In [1]:
# -*- coding: utf8 -*-
import codecs,glob
import features
import re
import pandas as pd
import numpy as np
import matplotlib as mplt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import itertools as it
import networkx as nx
import pickle,yaml
import random as rd
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

In [2]:
import unicodedata
def listerUnicode(chaine):
    result=[]
    comb=False
    prec=u""
    for char in chaine:
        print char,
        if unicodedata.combining(char):
            result.append(prec+char)
            prec=""
        else:
            result.append(prec)
            prec=char
    result.append(prec)
    return [r for r in result if r!=""]

In [3]:
ipaIn = listerUnicode(u"SZNêôârEHO")
ipaOut= [u"ʃ",u"ʒ",u"ŋ",u"ɛ̃",u"ɔ̃",u"ɑ̃",u"ʁ",u"ɛ",u"ɥ",u"ɔ"]
toipa = dict(zip(ipaIn, ipaOut))

S Z N ê ô â r E H O


In [4]:
def coderIPA(chaine,table=toipa):
    result=chaine
    for k in table:
        result=result.replace(k,table[k])
    return result

print coderIPA(u"SâZ piNpôN")

ʃɑ̃ʒ piŋpɔ̃ŋ


# Faire le tableau de paradigmes de la base de verbes

In [5]:
rep="/Volumes/BroadDisk/Copies-iMac-GB/2015-Data/"
dfGold=pd.read_pickle(rep+"MGC-171229-Verbes3.pkl")

In [14]:
dfLex3=dfGold[dfGold["freq"]>10000]
dfLex3.head()

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
40,a,a,NaN,V,3S,pi,avoir,7.167735e+11,2.076214e-02,pi3S
45,abaissa,abEsa,NaN,V,3S,ai,abaisser,2.640001e+08,7.647056e-06,ai3S
46,abaissai,abEsE,NaN,V,1S,ai,abaisser,1.700001e+07,4.924243e-07,ai1S
47,abaissaient,abEsE,"t""",V,3P,ii,abaisser,4.100005e+07,1.187612e-06,ii3P
50,abaissait,abEsE,"t""",V,3S,ii,abaisser,2.520003e+08,7.299467e-06,ii3S


In [8]:
dfPar3=pd.pivot_table(dfLex3, values="phono", index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
cases=dfPar3.columns[2:].tolist()
dfPar3.head()

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,NaN,abEsE,NaN,NaN,abEsEr,abEsa,NaN,abEs6rE,NaN,...,abEse,abEse,abEse,abEse,NaN,NaN,NaN,NaN,NaN,NaN
1,abandonner,abâdOnam,abâdOnE,NaN,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,...,abâdOne,abâdOne,abâdOne,abâdOne,NaN,NaN,NaN,abâdOn,abâdOn,abâdOn
2,abasourdir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,abazurdi,abazurdi,abazurdi,NaN,NaN,NaN,NaN,NaN,NaN
3,abattre,NaN,abati,NaN,NaN,abatir,abati,abatrô,abatrE,abatre,...,abaty,abaty,abaty,abaty,NaN,abat,NaN,abat,NaN,abat
4,abdiquer,NaN,abdikE,NaN,NaN,NaN,abdika,NaN,NaN,NaN,...,NaN,abdike,NaN,abdike,NaN,NaN,NaN,NaN,NaN,NaN


# Calculer la constante et la différence d'une paire de formes

In [9]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
    return (same,diff1,diff2,diff1+"_"+diff2)
#    return (diff1+"-"+diff2)

In [10]:
diff("abad","abod")

('ab.d', 'a.', 'o.', 'a._o.')

# Extraire le radical et le patron d'un paradigme verbal

In [11]:
def removeRad(forme,radical):
    result=""
    frontiere=False
    for nEl,element in enumerate(forme):
        if not frontiere and (nEl>=len(radical) or element!=radical[nEl]):
            frontiere=True
            result+=element
        elif frontiere and nEl<len(radical) and radical[nEl]!=".":
            result+="."
        elif frontiere:
            result+=element
    return result

def extractRad(formes):
    radical=formes[0]
    for forme in formes[1:]:
        radical=diff(radical,forme)[0]
    return radical

def makePatAlt(lPar,radical):
    patAlt={}
    for case in lPar:
        patAlt[case]=removeRad(lPar[case],radical)
    return patAlt

In [12]:
formes=u"m@ner mEn@r m@nôr mEn@rô".split(" ")
extractRad(formes)
removeRad(u"abEsâ",u'abEs')

u'\xe2'

# Calcul des patrons bruts

In [13]:
def patronLigne(row):
    patron={}
    formes=[]
    lPar={}
    patAlt={}
    for case in cases:
        if isinstance(row[case],unicode):
            formes.append(row[case])
            lPar[case]=row[case]
    if len(lPar)>1:
        radical=extractRad(formes)
        patAlt=makePatAlt(lPar,radical)
    return patAlt

In [15]:
dfPar3["patron"]=dfPar3.apply(lambda x: patronLigne(x),axis=1)
dfPar3.head()

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S,patron
0,abaisser,NaN,abEsE,NaN,NaN,abEsEr,abEsa,NaN,abEs6rE,NaN,...,abEse,abEse,abEse,NaN,NaN,NaN,NaN,NaN,NaN,"{u'pP': u'â', u'ppMS': u'e', u'ai3P': u'Er', u..."
1,abandonner,abâdOnam,abâdOnE,NaN,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,...,abâdOne,abâdOne,abâdOne,NaN,NaN,NaN,abâdOn,abâdOn,abâdOn,"{u'ii1P': u'jô', u'ii1S': u'E', u'pP': u'â', u..."
2,abasourdir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,abazurdi,abazurdi,abazurdi,NaN,NaN,NaN,NaN,NaN,NaN,"{u'ppMS': u'', u'inf': u'r', u'ppMP': u'', u'p..."
3,abattre,NaN,abati,NaN,NaN,abatir,abati,abatrô,abatrE,abatre,...,abaty,abaty,abaty,NaN,abat,NaN,abat,NaN,abat,"{u'ii1P': u'tjô', u'ii1S': u'tE', u'pP': u'tâ'..."
4,abdiquer,NaN,abdikE,NaN,NaN,NaN,abdika,NaN,NaN,NaN,...,abdike,NaN,abdike,NaN,NaN,NaN,NaN,NaN,NaN,"{u'ai3S': u'a', u'ii1S': u'E', u'ii3P': u'E', ..."


# Cacul du nombre de lexèmes par patron brut

In [28]:
dfPatronsTemp=dfPar3[["lexeme","patron"]]

In [29]:
dfPatrons=pd.concat([dfPatronsTemp.drop(['patron'], axis=1), dfPatronsTemp['patron'].apply(pd.Series)], axis=1)

In [31]:
dfPatrons["nbCases"]=dfPatrons.notnull().sum(axis=1)
dfPatrons.fillna("-",inplace=True)

In [35]:
dfPatrons.groupby(cases).agg({"lexeme":"count","nbCases":"first"}).sort_values('lexeme', ascending=False).head()

lexeme  \
ai1S ai2P ai2S ai3P ai3S fi1P fi1S fi2P fi2S fi3P fi3S ii1P ii1S ii2P ii2S ii3P ii3S inf is1P is1S is2P is2S is3P is3S pI1P pI2P pI2S pP pc1P pc1S pc2P pc2S pc3P pc3S pi1P pi1S pi2P pi2S pi3P pi3S ppFP ppFS ppMP ppMS ps1P ps1S ps2P ps2S ps3P ps3S           
-    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -   -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -        458   
                                                                                         -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -    -    -         -    -    -    -    -    -         69   
                                                                                     -   -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -         -         -    -    -    -    -    -         31   
                                                                                         -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -                        -    -    -    -    -    -         20   
                                                                                                                                                                                                     -                   -    -    -    -    -    -         20   

                                                                                                                                                                                                                                                        nbCases  
ai1S ai2P ai2S ai3P ai3S fi1P fi1S fi2P fi2S fi3P fi3S ii1P ii1S ii2P ii2S ii3P ii3S inf is1P is1S is2P is2S is3P is3S pI1P pI2P pI2S pP pc1P pc1S pc2P pc2S pc3P pc3S pi1P pi1S pi2P pi2S pi3P pi3S ppFP ppFS ppMP ppMS ps1P ps1S ps2P ps2S ps3P ps3S           
-    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -   -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -           1  
                                                                                         -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -    -    -         -    -    -    -    -    -           3  
                                                                                     -   -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -         -         -    -    -    -    -    -           3  
                                                                                         -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -                        -    -    -    -    -    -           6  
                                                                                                                                                                                                     -                   -    -    -    -    -    -           5

In [36]:
dfPatronsBruts=dfPatrons.groupby(cases).agg({"lexeme":"count","nbCases":"first"}).sort_values('nbCases', ascending=False).reset_index()

In [237]:
def compatibiliteComp(c1B,c2B,extra1="-",extra2="-"):
    '''
    c1B et c2B sont les éléments de patron locaux, 
    extra1 et extra2 sont les éléments ajoutés à gauche des patrons
    on ajoute les extras pendant la comparaison et on resépare les éléments à la fin
    '''
#    print c1B,c2B,extra1,extra2
    lMax=max(len(c1B),len(c2B))
    if extra1=="-":
        c1=c1B
    else:
        c1=extra1+c1B
    if extra2=="-":
        c2=c2B
    else:
        c2=extra2+c2B
    lPossible=True
    if c1!="-" and c2!="-":
        longueurAB=len(c1)
        extra=""
        if longueurAB<len(c2):
            extra=c2[:len(c2)-longueurAB]
        elif longueurAB>len(c2):
            longueurAB=len(c2)
            extra=c1[:len(c1)-longueurAB]
        result=extra
        c1Comp=c1[-longueurAB:]
        c2Comp=c2[-longueurAB:]
#        print c1Comp,c2Comp,longueurAB
        lCompatibilite=True
        for c in range(longueurAB):
            if c1Comp[c]==".":
                result+=c2Comp[c]
            elif c2Comp[c]==".":
                result+=c1Comp[c]
            elif c1Comp[c]==c2Comp[c]:
                result+=c1Comp[c]
            else:
                lPossible=False
                break
    elif c1=="-":
#        print "-",c2
        result=c2
    else:
#        print c1,"-"
        result=c1
    comp1=result[-len(c1B):]
    comp2=result[-len(c2B):]
    extra1=result[:-len(c1B)]
    extra2=result[:-len(c2B)]
    
    return lPossible,extra1,comp1,extra2,comp2
    

In [238]:
compatibiliteComp("b.ik","s.k","s","-")

(True, 's', 'bsik', 'sb', 'sik')

In [255]:
def compatibiliteAB(lDF,a,b):
    rowA=lDF.iloc[a]
    rowB=lDF.iloc[b]
    extra1="-"
    extra2="-"
    lPossible=True
    result=""
    patronAB=[]
    for case in cases:
#        print case,
        lPossible,extra1,comp1,extra2,comp2=compatibiliteComp(rowA[case],rowB[case],extra1,extra2)
        if not lPossible:
            break
        patronAB.append((comp1,comp2))
#         p1C=rowA[case]
#         p2C=rowB[case]
#         if p1C!="-" and p2C!="-":
#             longueurAB=len(p1C)
#             if longueurAB<len(p2C):
#                 extra2=p2C[:len(p2C)-longueurAB]
#             elif longueurAB>len(p2C):
#                 longueurAB=len(p2C)
#                 extra1=p1C[:len(p1C)-longueurAB]
#             p1Comp=p1C[-longueurAB:]
#             p2Comp=p2C[-longueurAB:]
#             print p1C,p2C
#             pComp=""
#             for c in range(longueurAB):
#                 if p1Comp[c]==".":
#                     pComp+=p2Comp[c]
#                 elif p2Comp[c]==".":
#                     pComp+=p1Comp[c]
#                 elif p1Comp[c]==p2Comp[c]:
#                     pComp+=p1Comp[c]
#                 else:
#                     lPossible=False
#             print lPossible, pComp
#             if not lPossible:
#                 break
#         elif p1=="-":
#             print "-",p2C
#             patronAB.append(p2C)
#         else:
#             print p1C,"-"
#             patronAB.append(p1C)
    result=";".join([extra1+comp1 for (comp1,comp2) in patronAB])
    if not lPossible:
#        print "Echec",result
        result=None
#    print "Compatible",a,b,result
    return result

In [256]:
compatibiliteAB(dfPatronsBruts,6,7)

# Faire un graphe dirigé pour les patrons
1. introduire tous les patrons de base
1. ajouter un lien vers le patronAB si le patron est différent pour A, pour B
1. recommencer avec les sommets

In [270]:
graphPatrons=nx.DiGraph()
def getSommets(lDiGraph):
    return [k for k,v in lDiGraph.out_degree().iteritems() if v==0]

[12, 4]

In [259]:
possibleUnions={}
for a,b in it.combinations(range(100),2):
    patronAB=compatibiliteAB(dfPatronsBruts,a,b)
    if patronAB not in possibleUnions:
        possibleUnions[patronAB]=[]
    possibleUnions[patronAB].append((a,b))
print
# for numU,u in enumerate(possibleUnions):
#     print numU,u
possibleUnions

{None: [(0, 1),
  (0, 2),
  (0, 3),
  (0, 4),
  (0, 5),
  (0, 6),
  (0, 7),
  (0, 8),
  (0, 9),
  (0, 10),
  (0, 11),
  (0, 12),
  (0, 13),
  (0, 14),
  (0, 15),
  (0, 16),
  (0, 17),
  (0, 18),
  (0, 19),
  (0, 20),
  (0, 21),
  (0, 22),
  (0, 23),
  (0, 24),
  (0, 25),
  (0, 26),
  (0, 27),
  (0, 28),
  (0, 29),
  (0, 30),
  (0, 31),
  (0, 32),
  (0, 33),
  (0, 34),
  (0, 35),
  (0, 36),
  (0, 37),
  (0, 38),
  (0, 39),
  (0, 40),
  (0, 41),
  (0, 42),
  (0, 43),
  (0, 44),
  (0, 45),
  (0, 46),
  (0, 47),
  (0, 48),
  (0, 49),
  (0, 50),
  (0, 51),
  (0, 52),
  (0, 53),
  (0, 54),
  (0, 55),
  (0, 56),
  (0, 57),
  (0, 58),
  (0, 59),
  (0, 60),
  (0, 61),
  (0, 62),
  (0, 63),
  (0, 64),
  (0, 65),
  (0, 66),
  (0, 67),
  (0, 68),
  (0, 69),
  (0, 70),
  (0, 71),
  (0, 72),
  (0, 73),
  (0, 74),
  (0, 75),
  (0, 76),
  (0, 77),
  (0, 78),
  (0, 79),
  (0, 80),
  (0, 81),
  (0, 82),
  (0, 83),
  (0, 84),
  (0, 85),
  (0, 86),
  (0, 87),
  (0, 88),
  (0, 89),
  (0, 90),
  (0, 91),
  

In [106]:
dfPatronsBruts[dfPatronsBruts["pi1S"].str.contains(ur"\.")]["pi1S"]
#dfPatronsBruts.rename(columns={"lexeme":"nbCases"})

31       9.
51       E.
52       E.
53       E.
54       E.
61       E.
66       o.
102      o.
113      E.
161      o.
171      E.
172      E.
174      9.
178      E.
179      E.
184      E.
203      o.
210      o.
240      E.
258      o.
259      o.
265      2.
283      o.
285      o.
286      o.
287      o.
294      9.
298      E.
301      E.
304      o.
       ... 
2468     E.
2549     o.
2556     E.
2601     o.
2604     o.
2636     E.
2771     o.
2787     o.
2856     E.
2860     E.
2865     o.
2892     2.
2904     o.
3026     E.
3111     E.
3137     o.
3213     o.
3238    9..
3249     o.
3263     E.
3264     E.
3319     E.
3386     2.
3628     o.
3631    o..
3695     o.
3841     o.
3858     E.
3943     o.
4192     o.
Name: pi1S, Length: 153, dtype: object